In [ ]:
import asyncio
from dotenv import dotenv_values
import uuid
import json
from azure.cosmos.aio import CosmosClient as AsyncClient
from azure.cosmos import PartitionKey
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Helper function to print the query stats
def print_stats(headers):
    time_taken = headers['x-ms-documentdb-query-metrics'].split(";")[0]
    ru_charge = float(headers['x-ms-request-charge'])*8
    cost =  (ru_charge/1000000)*0.25
    print("Query Execution Time (ms): ",time_taken.split("=")[1])
    print("RU charge for this Vector Search:", ru_charge)
    print(f"Cost (USD) for this Vector Search in Cosmos DB Serverless: ${cost:.9f}")

In [ ]:
env_name = "config.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

cosmos_100M_connection =  config['cosmos_100M_connection']
cosmos_100M_key = config['cosmos_100M_key']

In [ ]:
cosmos_100M_connection # Already defined
cosmos_100M_key # Already defined
cosmos_100M_database = "vector-benchmarking"
cosmos_100M_collection = "msturing-100M-8"

# Sample vector
This is a vector embedding from the Microsoft Turing dataset of embeddings. This was sampled from the dataset, and a small amount of noise was added.

In [ ]:

e = [-0.14259713888168335, -0.024914590641856194, 0.046193722635507584, 0.22360573709011078, -0.23203463852405548, 0.16737870872020721, 0.20103447139263153, 0.028627103194594383, -0.04843227565288544, 0.03130139037966728, 0.06655777245759964, -0.03377373144030571, 0.14429964125156403, 0.02467477321624756, -0.04912057891488075, -0.1553136706352234, -0.06603388488292694, 0.06882616877555847, 0.013733630068600178, 0.1496177315711975, -0.08906995505094528, 0.1425914466381073, 0.05974918231368065, -0.1144903153181076, -0.09413857758045197, -0.05027080327272415, 0.09943030774593353, -0.10074792802333832, -0.11043097823858261, 0.09908539801836014, -0.10460341721773148, -0.014216256327927113, 0.06218205764889717, 0.12143395841121674, -0.014290843158960342, -0.1561131626367569, 0.10752971470355988, 0.03346358984708786, -0.014386676251888275, 0.2015041559934616, -0.041845180094242096, 0.1352653056383133, 0.055085670202970505, -0.004726277198642492, 0.06408729404211044, 0.011137771420180798, 0.1759580820798874, 0.03425472974777222, -0.17353695631027222, -0.20622095465660095, -0.17467279732227325, -0.058571696281433105, 0.03347020968794823, 0.14000941812992096, -0.005757465027272701, -0.16064311563968658, -0.008287477307021618, 0.024134453386068344, 0.007042454555630684, -0.0977964699268341, 0.10146485269069672, 0.046062865805864334, -0.050479013472795486, -0.16915898025035858, 0.01924767531454563, 0.062267474830150604, -0.17726202309131622, 0.00631012674421072, 0.04163888096809387, 0.01714254543185234, 0.12430141121149063, -0.008242920972406864, 0.05079461634159088, -0.0164614450186491, -0.0711546316742897, 0.015482721850275993, 0.09131310880184174, 0.052265215665102005, 0.011293343268334866, 0.016989994794130325, -0.041206736117601395, -0.08246427774429321, -0.014159539714455605, -0.001703119371086359, 0.036216214299201965, 0.10411141067743301, -0.03630885109305382, 0.0001756631681928411, -0.11523693799972534, 0.08395029604434967, -0.004393506329506636, -0.006109242793172598, -0.18553389608860016, -0.04536907374858856, 0.18034495413303375, -0.06162665784358978, -0.14566700160503387, -0.1082751676440239, -0.13942785561084747, 0.04452257975935935 ]


# Vector Search at Scale with Azure Cosmos DB for NoSQL
Vector search on 100M Microsoft Turing embeddings.

In [ ]:
async def fetch_query_results(container, query, pk_range):
    return [item async for item in container.query_items(query=query,parameters=[{'name' : '@query_vector', 'value' : e}], partition_key_range_id=pk_range['id'], populate_query_metrics=True)]

In [ ]:
async def concurrent_query_sample(query):
    async with AsyncClient(url=cosmos_100M_connection, credential=cosmos_100M_key) as client:
        db_100M = client.get_database_client(cosmos_100M_database)
        container_100M = db_100M.get_container_client(cosmos_100M_collection)

        # Fetch the partition key range ids for the container
        pk_ranges = [pk_range async for pk_range in  container_100M.client_connection._ReadPartitionKeyRanges(container_100M.container_link)]
        # Queue up queries on a per-partition basis
        tasks = [fetch_query_results(container_100M, query, pk_range) for pk_range in pk_ranges]
        # Run them all and gather the results
        results = await asyncio.gather(*tasks)
        return results, client.client_connection.last_response_headers

In [ ]:
query = '''
        SELECT TOP 10 c.id, VectorDistance(c.embedding, @query_vector) as Score
        FROM c
        ORDER BY VectorDistance(c.embedding, @query_vector)
    '''

In [ ]:
# Run the query across all partitions
results, headers = await concurrent_query_sample(query)

# Merge and sort the results
flattened_results = [item for sublist in results for item in sublist]
sorted_results = sorted(flattened_results, key=lambda x: x["Score"])
print(json.dumps(sorted_results[:10],indent=4))
print_stats(headers)